In [1]:
from datetime import datetime

In [2]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

In [3]:
class Logger:
    current_time = TimeUTC()

In [4]:
Logger.current_time

'2022-02-08T01:37:11.807097'

In [5]:
l = Logger
l.current_time

'2022-02-08T01:37:22.292410'

In [7]:
from random import choice, seed

class Deck:
    @property
    def suit(self):
        return choice( ('Spade', 'Heart', 'Diamond', 'Club') )
    
    @property
    def card(self):
        return choice( tuple('23456789JQKA') + ('10',) )

In [8]:
d = Deck()

for _ in range(10):
    print(d.card, d.suit)

6 Heart
Q Heart
K Heart
10 Spade
3 Heart
A Club
2 Club
J Heart
5 Spade
10 Spade


In [10]:
class Choice:
    def __init__(self, *choices):
        self.choices = choices
        
    def __get__(self, instance, owner_class):
        return choice(self.choices)

In [11]:
class Deck:
    suit = Choice('Spade', 'Heart', 'Diamond', 'Club')
    card = Choice(*'23456789JQKA', '10')

In [12]:
d = Deck()
for _ in range(10):
    print(d.card, d.suit)

5 Diamond
3 Spade
9 Spade
J Spade
A Spade
4 Diamond
3 Diamond
7 Spade
9 Club
8 Diamond


In [13]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        print(f"__get__ called, self={self}, instance={instance}, owner_class={owner_class}")
        return datetime.utcnow().isoformat()

In [14]:
class Logger1:
    current_time = TimeUTC()
    
class Logger2:
    current_time = TimeUTC()

In [15]:
Logger1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x7fa2da12a940>, instance=None, owner_class=<class '__main__.Logger1'>


'2022-02-08T01:49:09.216662'

In [16]:
Logger2.current_time

__get__ called, self=<__main__.TimeUTC object at 0x7fa2da12a820>, instance=None, owner_class=<class '__main__.Logger2'>


'2022-02-08T01:50:12.055200'

In [17]:
l1 = Logger1()
print(hex(id(l1)))
l1.current_time

0x7fa2d9bd6d30
__get__ called, self=<__main__.TimeUTC object at 0x7fa2da12a940>, instance=<__main__.Logger1 object at 0x7fa2d9bd6d30>, owner_class=<class '__main__.Logger1'>


'2022-02-08T01:50:42.102319'

In [18]:
l2 = Logger1()
l2.current_time

__get__ called, self=<__main__.TimeUTC object at 0x7fa2da12a940>, instance=<__main__.Logger1 object at 0x7fa2da01d4c0>, owner_class=<class '__main__.Logger1'>


'2022-02-08T01:51:28.226873'

In [19]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return datetime.utcnow().isoformat()    

In [20]:
class Logger:
    current_time = TimeUTC()

In [21]:
Logger.current_time

In [23]:
l = Logger()
l.current_time

'2022-02-08T01:54:13.885535'

In [24]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print(f"__get__ called in {self}")
            return datetime.utcnow().isoformat() 
        
class Logger:
    current_time = TimeUTC()

In [25]:
l1 = Logger()
l2 = Logger()

l1.current_time, l2.current_time

__get__ called in <__main__.TimeUTC object at 0x7fa2d973f370>
__get__ called in <__main__.TimeUTC object at 0x7fa2d973f370>


('2022-02-08T01:57:13.947465', '2022-02-08T01:57:13.947563')

In [30]:
class Countdown:
    def __init__(self, start):
        self.start = start + 1
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        self.start -= 1
        return self.start
    
class ShuttleLaunch:
    countdown = Countdown(10)

In [31]:
s1 = ShuttleLaunch()
s2 = ShuttleLaunch()

In [32]:
s1.countdown

10

In [33]:
s2.countdown

9

In [35]:
class IntegerValue:
    def __set__(self, instance, value):
        print(f"__set__ called, instance={instance}, value={value}")
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ called from class')
        else:
            print(f"__get__ called, instance={instance}, owner_class={owner_class}")

In [36]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [38]:
Point2D.x

__get__ called from class


In [40]:
p = Point2D()
p.x

__get__ called, instance=<__main__.Point2D object at 0x7fa2d9f04d90>, owner_class=<class '__main__.Point2D'>


In [41]:
p.x = 100

__set__ called, instance=<__main__.Point2D object at 0x7fa2d9f04d90>, value=100


In [42]:
class IntegerValue:
    def __init__(self, name):
        self.storage_name = '_' + name
    
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        return getattr(instance, self.storage_name, None)

In [44]:
class Point2D:
    x = IntegerValue('x')
    y = IntegerValue('y')

In [46]:
p1, p2 = Point2D(), Point2D()

p1.x = 10.1
p1.y = 20.2
p1.__dict__

{'_x': 10, '_y': 20}

In [47]:
p2.__dict__

{}

In [72]:
# instances must be hashable, memory leak since theres always a ref to instance obj
class IntegerValue:
    def __init__(self):
        self.values = dict()
    
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        return self.values.get(instance, None)

In [73]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [74]:
p1, p2 = Point2D(), Point2D()
p1.x = 10.1
p2.x = 100.1

Point2D.x.values

{<__main__.Point2D at 0x7fa2d9728550>: 10,
 <__main__.Point2D at 0x7fa2d9728c10>: 100}

In [4]:
import weakref

In [81]:
# instance must be hashable
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
    
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self.values.get(instance, None)

In [82]:
class Point:
    x = IntegerValue()

In [83]:
p = Point()
print(hex(id(p)))

p.x = 100.1
Point.x.values.keyrefs()

0x7fa2da109370


[<weakref at 0x7fa2d9c4f3b0; to 'Point' at 0x7fa2da109370>]

In [84]:
del p

Point.x.values.keyrefs()

[]

In [89]:
# no way for keys to be removed when instance is deleted
class IntegerValue:
    def __init__(self):
        self.values = dict()
    
    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self.values.get(id(instance), None)

In [90]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [91]:
p = Point(10.1)
p.x = 20.2

id(p), Point.x.values

(140337410277184, {140337410277184: 20})

In [5]:
# cannot be used with instances which utilize slots
class IntegerValue:
    def __init__(self):
        self.values = dict()
    
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self.values[id(instance)][1]
        
    def _remove_object(self, weak_ref):
        print(f"removing dead entry for {weak_ref}")
        reverse_lookup = [k for k, v in self.values.items()
                         if v[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]

In [6]:
class Point:
    x = IntegerValue()

In [7]:
p1, p2 = Point(), Point()

p1.x, p2.x = 10.1, 20.2
p1.x, p2.x

(10, 20)

In [116]:
Point.x.values

{140337414114272: (<weakref at 0x7fa2d956a680; to 'Point' at 0x7fa2d9b78be0>,
  10),
 140337414111392: (<weakref at 0x7fa2d9985a40; to 'Point' at 0x7fa2d9b780a0>,
  20)}

In [8]:
del p1

removing dead entry for <weakref at 0x7f371677e810; dead>


In [9]:
Point.x.values

{139874576844832: (<weakref at 0x7f3716757b80; to 'Point' at 0x7f3716774820>,
  20)}

In [10]:
class ValidString:
    def __init__(self, min_len, max_len):
        self.data = dict()
        self._min_len = min_len
        self._max_len = max_len
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError("Value must be 'str'")
        if len(value) < self._min_len:
            raise ValueError(f"Value must be at least {self._min_len} characters")
        if len(value) > self._max_len:
            raise ValueError(f"Value must be less than {self._max_len} characters")
            
        weak_ref = weakref.ref(instance, self._finalize_instance)
        self.data[id(instance)] = (weak_ref, value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.data.get(id(instance))[1]
        
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [k for k, v in self.data.items()
                         if v[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [11]:
class Person:
    __slots__ = '__weakref__',
    
    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)
    
    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )

In [13]:
class BankAccount:
    __slots__ = '__weakref__',
    
    account_number = ValidString(5, 255)
    
    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

In [14]:
p1 = Person()
p1.first_name = ''

ValueError: Value must be at least 1 characters

In [15]:
p2 = Person()

In [16]:
p1.first_name, p1.last_name = 'Guy', 'Smith'
p2.first_name, p2.last_name = 'John', 'Doe'

In [17]:
b1, b2 = BankAccount(), BankAccount()
b1.account_number = '1000100110011'
b2.account_number = '200222202202'

In [18]:
p1.first_name

'Guy'

In [19]:
b2.account_number

'200222202202'

In [20]:
Person.first_name.data

{139874560305232: (<weakref at 0x7f3714e29a40; to 'Person' at 0x7f37157ae850>,
  'Guy'),
 139874557897504: (<weakref at 0x7f37152dc7c0; to 'Person' at 0x7f3715562b20>,
  'John')}

In [24]:
BankAccount.account_number.data

{139874553388048: (<weakref at 0x7f37152dc770; to 'BankAccount' at 0x7f3715115c10>,
  '1000100110011'),
 139874553385504: (<weakref at 0x7f3715799a40; to 'BankAccount' at 0x7f3715115220>,
  '200222202202')}

In [2]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__ called, owner={owner_class}, property={property_name}")
        self.property_name = property_name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print(f"__get__ called for property: {self.property_name} of instance: {instance}")
            
    
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__ called, owner=<class '__main__.Person'>, property=first_name
__set_name__ called, owner=<class '__main__.Person'>, property=last_name


In [4]:
p = Person()
p.first_name

__get__ called for property: first_name of instance: <__main__.Person object at 0x7f8a2c3f09a0>


In [9]:
class ValidString:
    def __init__(self, min_len):
        self.min_len = min_len
        
    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f"{self.property_name} must be 'str'")
        if len(value) < self.min_len:
            raise ValueError(f"{self.property_name} must be more than {self.min_len} characters")
        
        instance.__dict__[self.property_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            instance.__dict__.get(self.property_name, None)

In [10]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [11]:
p = Person()

try:
    p.first_name = 'AJ'
    p.last_name = 'X'
except ValueError as exc:
    print(exc)

last_name must be more than 2 characters


In [12]:
p.first_name, p.last_name = 'AJ', 'Smith'
p.__dict__

{'first_name': 'AJ', 'last_name': 'Smith'}

In [13]:
class IntegerValue:
    def __set__(self, instance, value):
        print('__set__ called')
        
    def __get__(self, instance, owner_class):
        print('__get__ called')
        
class Point:
    x = IntegerValue()

In [14]:
p = Point()
p.x = 100

__set__ called


In [15]:
p.x

__get__ called


In [16]:
p.__dict__

{}

In [17]:
p.__dict__['x'] = 'hello'
p.x

__get__ called


In [18]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        print('__get__ called')
        
class Logger:
    current_time = TimeUTC()

In [19]:
l = Logger()
l.current_time

__get__ called


In [20]:
print(l.__dict__)
l.__dict__['current_time'] = 'hello'
l.__dict__

{}


{'current_time': 'hello'}